## Data Processing Using Apache Spark on Openshift

This notebook server is hosted on the OpenShift platform which provides a dedicated notebook server for each individual user. The platform takes care of provisioning the cluster resources including the allocation related to storage resources.

### Manually set variables in notebook mode

In [12]:
import os
os.environ['S3_ENDPOINT'] = "http://minio-ml-workshop:9000"

###  Prepare S3 URL
Define a function that will convert an S3 URL into URL that works with MINIO

In [13]:
import os, socket
from urllib.parse import urlparse

# Get the S3 URL information and use it in Spark Context
# NOTE: S3 Hadoop API for spark does not work with domain name, use IP address instead
def domain_to_ip(url):
    domain = urlparse(url).netloc.split(":")[0]
    ip_address = socket.gethostbyname(domain)
    ip_url = url.replace(domain, ip_address)
    return ip_url

###  Connect to Spark Cluster provided by OpenShift Platform
Using the given spark_util library, create a Spark session that connects to a Spark cluster dedicated for this notebook. You may add additional Spark submit arguments in the second argument of spark_util.getOrCreateSparkSession() such as additional packages and or override some configuration items.

In [14]:
import spark_util

submit_args = f"--conf spark.hadoop.fs.s3a.endpoint={domain_to_ip(os.environ['S3_ENDPOINT_URL'])} \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--conf spark.hadoop.fs.s3a.multipart.size=104857600 \
--conf spark.jars.ivy=/tmp \
--packages org.apache.hadoop:hadoop-aws:3.2.0"
# ,com.amazonaws:aws-java-sdk:1.11.968"

spark = spark_util.getOrCreateSparkSession("Customer Churn Ingest", submit_args)

Initializing environment variables for Spark
Cluter name: spark-cluster-rbrigoli
PYSPARK_SUBMIT_ARGS: --conf spark.hadoop.fs.s3a.endpoint=http://172.30.29.255:9000 --conf spark.hadoop.fs.s3a.access.key=minio --conf spark.hadoop.fs.s3a.secret.key=minio123 --conf spark.hadoop.fs.s3a.path.style.access=true --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem --conf spark.hadoop.fs.s3a.multipart.size=104857600 --conf spark.jars.ivy=/tmp --packages org.apache.hadoop:hadoop-aws:3.2.0 --master spark://spark-cluster-rbrigoli:7077 pyspark-shell 
Driver IP address: 10.130.3.188
Creating a spark session...
Spark session created


###  Create dataframes from CSV files

Using Spark, read the CSV filed from S3 storage and load them as Spark dataframes.

In [15]:
dataFrame_Customer = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/Customer-Churn_P1.csv")
dataFrame_Customer.printSchema()

dataFrame_Products = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/Customer-Churn_P2.csv")
dataFrame_Products.printSchema()

root
 |-- customerID: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)

root
 |-- customerID: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



### Join dataframes
Perform a full outer join on two dataframes using ```customerID``` as key

In [16]:

dataFrom_All = dataFrame_Customer.join(dataFrame_Products, "customerID", how="full")
dataFrom_All.printSchema()

root
 |-- customerID: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



###  Push the prepared data to the object storage and stop the Spark application
Write the joined dataframe to an S3 bucket. Because this is last step of our data preparation, we don't need the Spark cluster anymore. We will stop the Spark context which will remove the Spark application from the cluster.

<span style="color:red">Note: Change this value of user_id to your assigned username (something in the range user1 ... user30)</span>. 

In [17]:
user_id = "user_ross_parquet_single"
file_location = "s3a://data/full_data_parquet" + user_id
dataFrom_All.repartition(1).write.parquet(file_location + "/data.parquet")

In [18]:
spark.stop()